In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install git+https://github.com/facebookresearch/fastText

  Cloning https://github.com/facebookresearch/fastText to /tmp/pip-req-build-huldo844
  Running command git clone -q https://github.com/facebookresearch/fastText /tmp/pip-req-build-huldo844
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3013669 sha256=9afe3d39916bcf8b015cde0613f3cfcd1c13a756b3ded1a9bd650953373d4d30
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ivk14rz/wheels/72/f6/b9/3a859d37e24c73635e6c6af085ad99829e8b2ab1bd965b15a9
Successfully built fasttext


In [ ]:
#%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [ ]:
!ls /content/drive/My\ Drive/Code/data_aumentation_for_author_profiling

figures  notebooks  word_level_da


In [ ]:
!ls /content/drive/My\ Drive/Data/depresion/2019/train

class_distpng		   original_prep_ground_truth.txt
golden_truth_filtered.txt  prep_chunks
golden_truth.txt	   prep_chunks_filtered
ground_truth.txt	   train_golden_truth_joined.txt
original_prep


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Code/data_aumentation_for_author_profiling/')

In [ ]:

from datetime import datetime
from word_level_da import utils
from word_level_da.preprocessing.load_data import Dataset

In [ ]:
  epochs= 20      
  layers=1
  nodes=256
  dim=300
  label_pos=1
  len_doc=64
  key="depresion19_drive"
  LOAD_EMB=True
  fast_file='cc.en.300.bin'
  

In [ ]:
if LOAD_EMB:
  import fasttext.util
  fasttext.util.download_model('en', if_exists='ignore')  # English
  ft = fasttext.load_model('cc.en.300.bin')

Buffered data was truncated after reaching the output size limit.

In [ ]:
from word_level_da.classifier.train_sequence_model import seq_model
import numpy as np

In [ ]:
data = Dataset(key=key, doc_len=len_doc, min_len=int(len_doc / 2), chunking=True, remove_end=True)

In [ ]:
training, test=data.get_train_test(return_ids=True)

In [ ]:
bi_gru=seq_model(weights_path="/content/drive/My Drive/Code/datadata_aumentation_for_author_profiling/weights/", 
                 static=False, load_all_vectors=LOAD_EMB,
                 ids_labels=dict.fromkeys(test[2]).keys(), original_labels=test[3][0])

In [ ]:
n_docs=[i for i in range(1,2)]
ratio=0
method="Rel_1"
model="rnn-fixed"
umbral=0.5
score_m="avg"
q=75

In [ ]:
fold=0
n=0
#ratio=0
r=0.2

In [ ]:
scores=[]
AUGMENTED=False

In [ ]:
!ls "/content/drive/My Drive/Code/data_aumentation_for_author_profiling/word_level_da/obj/"

WE_depresion19.pkl


In [ ]:
for n in n_docs:
      
      prefix=method+str(n)
      
      if AUGMENTED:
        docs, l_docs= get_augmented_data(max_l=len_doc, min_l=int(len_doc/2), r_end=True, 
                                                         augmentation_method=method, n=n)
        new_training=np.append(training[0],docs)
        new_labels=np.append( training[1], l_docs)
      else:
        new_training=training[0]
        new_labels=training[1]

      info=bi_gru.buil_model(((new_training, new_labels), (test[0], test[1])), layers=1, nodes=nodes, embedding_dim=dim, dropout_rate=r, TOP_K=150000, 
                                pretrained=True, embedding_trainable=False, bidirectional=True, 
                          seq_len=len_doc, emb_file=fast_file, class_imbanlance=True, algo=model,
                          kernel_size=None, vocab_dir="/content/drive/My Drive/Code/data_aumentation_for_author_profiling/word_level_da/obj",
                           key=key.split("_")[0])
      info[2]=umbral
      for i in range(10):
        score=bi_gru.train_model(1e-3, epochs=epochs, patience=3, batch_size=1024, load_weights=False, save_weigths=False, 
                                                weights_name=key.split("_")[0]+prefix+model+str(i)+".h5",
                                  ad_data=(test[3]), monitor_measure="val_loss", validation=True,
                                 umbral=umbral,  score_method=score_m, q=q)
        
        final_info=[method,n,i ]
        final_info+=info+list(score)
        scores.append(final_info)
        print(final_info)


In [ ]:
for score in scores:
  print(score)

In [ ]:
bi_gru.save_predictions("/content/drive/My Drive/Code/data_aumentation_for_author_profiling/predictions/", "predictions_"+model+"_"+method)    
logger.info("Finish Time: %s", datetime.now())